<a href="https://colab.research.google.com/github/rhozon/Banca-FAE/blob/master/Scenarios_x_Real_(backtesting_portfolio).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting Scenarios vs Realizado (Últimos 22 Dias)

In [1]:
#@title 1) Instalação dos pacotes necessários

install.packages(
  c(
    "tidyverse",
    "tsibble",
    "fpp3",
    "quantmod",
    "timetk",
    "openxlsx",
    "IRdisplay"
  ),
  repos = "https://cloud.r-project.org"
)


Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘shape’, ‘future.apply’, ‘SQUAREM’, ‘diagram’, ‘lava’, ‘numDeriv’, ‘quadprog’, ‘prodlim’, ‘globals’, ‘listenv’, ‘parallelly’, ‘BH’, ‘distributional’, ‘progressr’, ‘ggdist’, ‘clock’, ‘gower’, ‘hardhat’, ‘ipred’, ‘sparsevctrs’, ‘furrr’, ‘lazyeval’, ‘crosstalk’, ‘warp’, ‘colorspace’, ‘fracdiff’, ‘lmtest’, ‘tseries’, ‘urca’, ‘RcppArmadillo’, ‘RcppRoll’, ‘future’, ‘anytime’, ‘fable’, ‘fabletools’, ‘feasts’, ‘tsibbledata’, ‘xts’, ‘zoo’, ‘TTR’, ‘recipes’, ‘rsample’, ‘plotly’, ‘padr’, ‘slider’, ‘timeDate’, ‘forecast’, ‘tsfeatures’




In [2]:
#@title 2) Carregamento dos pacotes

suppressPackageStartupMessages({
  library(tidyverse)
  library(tsibble)
  library(fpp3)
  library(quantmod)
  library(timetk)
  library(openxlsx)
  library(IRdisplay)
  library(tsibble)
  library(timetk)
  library(tidyr)

})


In [3]:

#@title 3) Definição dos tickers e download dos preços (corrigido)
tickers <- c("ZC=F","ZO=F","KE=F","GF=F","ZS=F","ZM=F","ZL=F")

# 1) Baixa cada série como xts e guarda numa lista
prices_list <- lapply(tickers, function(tk){
  quantmod::getSymbols.yahoo(
    tk,
    from       = "2019-01-01",
    auto.assign= FALSE
  )[,4]   # só a coluna de Adjusted/Close
})

# 2) Combina todas as séries num único objeto xts
portfolioPrices <- do.call(merge, prices_list)
colnames(portfolioPrices) <- c("corn","wheat","kcwheat",
                               "feeder","soymeal","soyf","soybeans")

# 3) Remove datas com qualquer NA
portfolioPrices <- na.omit(portfolioPrices)


Warning message:
“ZC=F contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”
Warning message:
“ZO=F contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”
Warning message:
“KE=F contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”
Warning message:
“GF=F contains missing values. Some functions will not work if objects contain missing values in the middle of the series. Consider using na.omit(), na.approx(), na.fill(), etc to remove or replace them.”
Warning message:
“ZS=F contains missing values. Some functions will not work if objects contain missing values i

In [4]:

#@title 4) Cálculo de log-retornos e conversão em tsibble (corrigido)


# 1) calcula retornos log
log_returns_xts <- diff(log(portfolioPrices))
log_returns_xts <- na.omit(log_returns_xts)

# 2) transforma em tibble com coluna date
log_returns <- tk_tbl(
  log_returns_xts,
  preserve_index = TRUE,
  rename_index   = "date"
)

# 3) pivot para long format
log_returns <- log_returns %>%
  pivot_longer(
    cols      = -date,
    names_to  = "asset",
    values_to = "ret"
  )

# 4) converte para tsibble
tsbl_returns <- log_returns %>%
  as_tsibble(key = asset, index = date)

# checa resultado
tsbl_returns |> group_by(asset) |> slice_tail(n = 3)



date,asset,ret
<date>,<chr>,<dbl>
2025-05-16,corn,-0.0112108797
2025-05-19,corn,0.0089787360
2025-05-20,corn,0.0155213759
2025-05-16,feeder,0.0022765267
2025-05-19,feeder,-0.0039661444
2025-05-20,feeder,0.0010986175
2025-05-16,kcwheat,-0.0224943679
2025-05-19,kcwheat,0.0120280497
2025-05-20,kcwheat,0.0254971319


In [6]:

#@title 5) Ajuste do modelo com regressão com quebras de tendência

models_all <- tsbl_returns |>
  model(
    Regr_Quebras = TSLM(ret ~ trend(knots = c(2020, 2022, 2024)))
  )



In [7]:

#@title 6) Definir datas de teste (últimas 22 datas de trading) e gerar cenários alinhados
library(dplyr); library(tsibble)

last_dates <- tsbl_returns |>
  distinct(date) |>
  arrange(date) |>
  tail(22) |>
  pull(date)

test_data <- tsbl_returns |>
  filter(date %in% last_dates)

sims_all <- models_all |>
  generate(
    new_data  = test_data,
    times     = 3,
    bootstrap = TRUE
  )



In [8]:
#@title 7) Extrair os retornos realizados no mesmo período de teste
library(tidyr)

realized_df <- test_data |>
  select(date, asset, ret) |>
  pivot_wider(
    names_from  = asset,
    values_from = ret
  )

In [9]:
#@title 8) Exportação para Excel: cenários e realizados
wb <- createWorkbook()
date_style <- createStyle(numFmt = "[$-pt-BR]ddd dd/mm/yyyy")
dec6_style <- createStyle(numFmt = "0.000000")

# Abas de cenários
for (c in sort(unique(sims_all$.rep))) {
  df_c <- sims_all |>
    filter(.rep == c) |>
    select(date, asset, .sim) |>
    pivot_wider(names_from = asset, values_from = .sim)
  sheet <- paste0("cenario_", c)
  addWorksheet(wb, sheet)
  writeData(wb, sheet, df_c)
  addStyle(wb, sheet, date_style, cols = 1, rows = 2:(nrow(df_c)+1), gridExpand = TRUE)
  num_cols <- which(sapply(df_c, is.numeric))
  addStyle(wb, sheet, dec6_style, cols = num_cols, rows = 2:(nrow(df_c)+1), gridExpand = TRUE)
  setColWidths(wb, sheet, cols = 1:ncol(df_c), widths = "auto")
}

# Aba de realizados
addWorksheet(wb, "realizados")
writeData(wb, "realizados", realized_df)
addStyle(wb, "realizados", date_style, cols = 1, rows = 2:(nrow(realized_df)+1), gridExpand = TRUE)
num_cols2 <- which(sapply(realized_df, is.numeric))
addStyle(wb, "realizados", dec6_style, cols = num_cols2, rows = 2:(nrow(realized_df)+1), gridExpand = TRUE)
setColWidths(wb, "realizados", cols = 1:ncol(realized_df), widths = "auto")

output_file <- "cenario_vs_realizado_22dias.xlsx"
saveWorkbook(wb, output_file, overwrite = TRUE)